In [94]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json
import csv
import random
from datetime import datetime,timedelta

In [95]:
#Iniciar a sessao Spark
spark = SparkSession.builder \
    .appName("EmpresaYAnaliseDeDados") \
    .getOrCreate()

In [96]:
# Gerar dados de vendas
def gerar_dados_vendas():
    produtos = ["Notebook", "Smartphone", "Tablet", "Monitor", "Teclado", "Mouse", "Impressora", "Cadeira", "Mesa", "Estante"]
    with open("dados_vendas.csv", "w", newline='') as arquivo:
        escritor = csv.writer(arquivo)
        escritor.writerow(["id_venda", "id_produto","produto", "quantidade", "preco", "data"])
        for i in range(1000):
            id_produto = random.randint(1, 10)
            escritor.writerow([
                i,
                id_produto,
                produtos[id_produto - 1],
                random.randint(1, 5),  # Quantidade entre 1 e 5
                "{:2f}".format(random.uniform(100.0, 1000)),  # Preço unitário entre 100 e 2000
                datetime.now() - timedelta(days=random.randint(0, 365))  # Data aleatória nos últimos 365 dias
            ])
   

In [97]:
# Gerar dados de comportamento do cliente
def gerar_dados_comportamento_cliente():
    acoes = ["visualizacao", "adicao_ao_carrinho", "compra", "avaliacao"]
    dados = []
    for i in range(1000):
        dados.append({
            "id_cliente": random.randint(1, 100),
            "acao": random.choice(acoes),
            "id_produto": random.randint(1, 10),
            "data_hora": (datetime.now() - timedelta(minutes =random.randint(0, 1440))).strftime("%Y-%m-%d %H:%M:%S")
        })
    with open("dados_comportamento_cliente.json", "w") as arquivo:
        json.dump(dados, arquivo)
       

In [98]:
# Gerar dados de Estoque
def gerar_dados_estoque():
    produtos = ["Notebook", "Smartphone", "Tablet", "Monitor", "Teclado", "Mouse", "Impressora", "Cadeira", "Mesa", "Estante"]
    with open("dados_estoque.csv", "w", newline='') as arquivo:
        escritor = csv.writer(arquivo)
        escritor.writerow(["id_produto", "nome_produto", "quantidade", "ultima_atualizacao" ])
        for i in range(1,6):
            escritor.writerow([
                i ,
                produtos[i - 1],
                random.randint(0, 1000),  # Quantidade de estoque entre 0 e 100
                datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Data de registro
            ])

In [99]:
# Gerar todos os dados
gerar_dados_vendas()
gerar_dados_comportamento_cliente()
gerar_dados_estoque()

In [100]:
# Carregar os dados de vendas

df_vendas = spark.read.csv("dados_vendas.csv", header=True, inferSchema=True)

# Carregar os dados de comportamento do cliente

df_comportamento = spark.read.json("dados_comportamento_cliente.json")

# Carregar os dados de estoque

df_estoque = spark.read.csv("dados_estoque.csv", header=True, inferSchema=True)



In [101]:
df_comportamento.show()

+------------------+-------------------+----------+----------+
|              acao|          data_hora|id_cliente|id_produto|
+------------------+-------------------+----------+----------+
|adicao_ao_carrinho|2025-07-26 20:57:20|        54|        10|
|adicao_ao_carrinho|2025-07-27 06:54:20|        39|         8|
|            compra|2025-07-27 01:44:20|        84|         5|
|         avaliacao|2025-07-26 10:38:20|        94|        10|
|         avaliacao|2025-07-27 00:58:20|        76|         4|
|         avaliacao|2025-07-27 03:48:20|         9|         4|
|      visualizacao|2025-07-26 10:58:20|        52|         8|
|         avaliacao|2025-07-26 12:13:20|        85|         7|
|adicao_ao_carrinho|2025-07-27 05:23:20|        89|         6|
|      visualizacao|2025-07-27 00:57:20|        87|         1|
|      visualizacao|2025-07-26 08:08:20|        58|         9|
|            compra|2025-07-27 03:52:20|        32|         9|
|            compra|2025-07-26 23:35:20|        57|    

In [102]:
# Processar os dados de vendas

resumo_vendas = df_vendas.groupBy("id_produto","produto")\
    .agg(
        sum("quantidade").alias("quantidade_total_vendida"),
        sum(col("quantidade") * col("preco")).alias("receita_total")
    )


In [103]:
df_estoque.show()

+----------+------------+----------+-------------------+
|id_produto|nome_produto|quantidade| ultima_atualizacao|
+----------+------------+----------+-------------------+
|         1|    Notebook|        79|2025-07-27 07:18:20|
|         2|  Smartphone|       336|2025-07-27 07:18:20|
|         3|      Tablet|        69|2025-07-27 07:18:20|
|         4|     Monitor|       859|2025-07-27 07:18:20|
|         5|     Teclado|       176|2025-07-27 07:18:20|
+----------+------------+----------+-------------------+



In [104]:
# Processar dados de comportamento do cliente
resumo_comportamento_cliente = df_comportamento.groupBy("id_produto", "acao") \
    .count() \
    .groupBy("id_produto") \
    .agg(
        sum(when(col("acao") == "visualizacao", col("count")).otherwise(0)).alias("visualizacoes"),
        sum(when(col("acao") == "adicao_ao_carrinho", col("count")).otherwise(0)).alias("adicoes_ao_carrinho"),
        sum(when(col("acao") == "compra", col("count")).otherwise(0)).alias("compras"),
        sum(when(col("acao") == "avaliacao", col("count")).otherwise(0)).alias("avaliacoes")  
    )

In [105]:
#  Processar dados de estoque

produtos_baixo_estoque = df_estoque.filter(col("quantidade") < 100)

# Juntar dados de vendas e estoque

vendas_estoque = resumo_vendas.join(df_estoque, resumo_vendas.id_produto == df_estoque.id_produto, "left")



#Calcular produtos mais vendidos com baixo estoque
mais_vendido_baixo_estoque = vendas_estoque.filter(col("quantidade") < 100)\
    .orderBy(col("quantidade_total_vendida").desc())\
    .limit(5)




# Analisar comportamento do cliente para produtos mais vendidos

produtos_top = resumo_vendas.orderBy(col("quantidade_total_vendida").desc()).limit(5)
comportamento_produtos_top = resumo_comportamento_cliente.join(produtos_top, resumo_comportamento_cliente.id_produto == produtos_top.id_produto, "inner")





In [107]:
# Calcular taxa de conversao (visualizações para compras)

taxa_conversao = resumo_comportamento_cliente.withColumn(
    "taxa_conversao",
    col("compras") / (col("visualizacoes") + 0.1)  # para evitar divisão por zero
)


print("Top 5 produtos mais vendidos com baixo estoque:")
mais_vendido_baixo_estoque.show()

print("\n Comportamento dos clientes para os produtos mais vendidos:")
comportamento_produtos_top.show()

print("\nTaxa de conversão por produtos:")
taxa_conversao.orderBy(col("taxa_conversao").desc()).show()



# Salvar os resultados em arquivos CSV
'''
mais_vendido_baixo_estoque.write.csv("mais_vendido_baixo_estoque.csv", header=True)
comportamento_produtos_top.write.csv("comportamento_produtos_top.csv", header=True)
taxa_conversao.write.csv("taxa_conversao.csv", header=True) 

'''

Top 5 produtos mais vendidos com baixo estoque:
+----------+--------+------------------------+------------------+----------+------------+----------+-------------------+
|id_produto| produto|quantidade_total_vendida|     receita_total|id_produto|nome_produto|quantidade| ultima_atualizacao|
+----------+--------+------------------------+------------------+----------+------------+----------+-------------------+
|         3|  Tablet|                     333|182682.12400399995|         3|      Tablet|        69|2025-07-27 07:18:20|
|         1|Notebook|                     262|152247.23004300005|         1|    Notebook|        79|2025-07-27 07:18:20|
+----------+--------+------------------------+------------------+----------+------------+----------+-------------------+


 Comportamento dos clientes para os produtos mais vendidos:
+----------+-------------+-------------------+-------+----------+----------+----------+------------------------+------------------+
|id_produto|visualizacoes|adicoe

'\nmais_vendido_baixo_estoque.write.csv("mais_vendido_baixo_estoque.csv", header=True)\ncomportamento_produtos_top.write.csv("comportamento_produtos_top.csv", header=True)\ntaxa_conversao.write.csv("taxa_conversao.csv", header=True) \n\n'